# Dataset Statistics Analysis

This notebook performs an initial overview of the dataset, providing basic statistics about the cell lines, drugs, and features in both the **GDSC** and **final datasets**. It will also include basic counts and summaries of the unique values and distribution in the dataset.

## Steps:
1. **Load datasets**: Load the GDSC, single-cell, and final processed datasets.
2. **Dataset statistics**: Calculate key metrics for the data.
3. **Save the results**: Save all statistics to CSV files for future use.


In [2]:
import os
import polars as pl

# Create a directory to store statistics
output_dir = "statistics"
os.makedirs(output_dir, exist_ok=True)

In [3]:
# Load datasets
gdsc_bulk = pl.read_parquet("../../data/gdsc/gdsc_final_cleaned.parquet").select([
    "SANGER_MODEL_ID", "DRUG_ID", "LN_IC50"
])
sc_data = pl.read_parquet("../../data/sc_data/rnaseq_all_data.parquet").select([
    "model_id", "gene_symbol", "fpkm"
])
final_data = pl.read_parquet("../../data/pseudo_bulk/gdsc_single_cell_aligned.parquet")

## 2. Compute Summary Statistics
This includes dataset dimensions, number of unique elements (genes, cell lines, drugs), and shape of the final modeling matrix.


In [4]:
stats_dict = {
    "🧬 Unique Genes in Raw Single-Cell": sc_data.select(pl.col("gene_symbol").n_unique()).item(),
    "🧬 Unique Cell Lines in Single-Cell": sc_data.select(pl.col("model_id").n_unique()).item(),
    "🧪 Total (Cell Line, Drug) Pairs in GDSC": gdsc_bulk.shape[0],
    "🧬 Total Cell Lines in GDSC": gdsc_bulk.select(pl.col("SANGER_MODEL_ID").n_unique()).item(),
    "💊 Total Unique Drugs in GDSC": gdsc_bulk.select(pl.col("DRUG_ID").n_unique()).item(),
    "📉 Features After PCA (Principal Components)": final_data.shape[1] - 3,  # remove ID, DRUG_ID, LN_IC50
    "📈 Cell Lines in Final Dataset": final_data.select(pl.col("SANGER_MODEL_ID").n_unique()).item(),
    "📈 Drugs in Final Dataset": final_data.select(pl.col("DRUG_ID").n_unique()).item(),
    "📊 (Cell Line, Drug) Pairs in Final Dataset": final_data.shape[0],
}


# Convert to Polars for CSV export
stats_df = pl.DataFrame([{"Metric": k, "Value": v} for k, v in stats_dict.items()])
stats_df.write_csv(f"{output_dir}/dataset_statistics.csv")

print("📁 Saved summary to 'statistics/dataset_statistics.csv'")
stats_df


📁 Saved summary to 'statistics/dataset_statistics.csv'


Metric,Value
str,i64
"""🧬 Unique Genes in Raw Single-C…",37601
"""🧬 Unique Cell Lines in Single-…",1363
"""🧪 Total (Cell Line, Drug) Pair…",571985
"""🧬 Total Cell Lines in GDSC""",971
"""💊 Total Unique Drugs in GDSC""",621
"""📉 Features After PCA (Principa…",37602
"""📈 Cell Lines in Final Dataset""",971
"""📈 Drugs in Final Dataset""",621
"""📊 (Cell Line, Drug) Pairs in F…",571985


### Conclusion

The dataset statistics have been calculated and saved successfully. You now have an overview of the number of cell lines, drugs, and the general structure of the dataset across GDSC, single-cell, and final datasets. This forms a foundation for further analysis and modeling.

- **Next Steps**: Proceed to further explore the distribution of IC50 values and correlation of genes with IC50.
